#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    > I have seen the flask app work with our dedicated backends.
    2. SQLAlchemy db object


In [1]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User: User is defining the template for users, it's a class definition template. This template is used to create objects (OOP). 
    - db.Model inheritance: Inheritance refers to how the class is called for the objects, and the objects inherits the properties and capabilities of the template. This in turn can help manipulate data in said database.
    - _init_ method: _init_ is what is used to derive/create an object from a class. 
    - ```@property```, ```@<column>.setter```: getter and setter methods. these are used to help solidify and add attributes to the template used for object creation.
    - create, read, update, delete methods:
        - Create: method that creates records in the database.
        - Read: method that helps with reading the data in the database.
        - Update: method that updates the things inside of the database.
        - Delete. method that deletes and terminates records in the database.

In [2]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()): # 123qwerty and datetime.today() used as defaults in case user does not define.
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [3]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Created new uid toby
Created new uid niko
Created new uid lex
Created new uid whit
Created new uid indi
Created new uid raven


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
    2. user.password

In [4]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [5]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Invalid date 2001-02-31 require YYYY-mm-dd, date defaulted to 03-16-2023
Created
 {'id': 7, 'name': 'hi', 'uid': '33', 'dob': '03-16-2023', 'age': 0}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [6]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-16-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-16-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-16-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': 'hi', 'uid': '33', 'dob': '03-16-2023', 'age': 0}]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

In [7]:
# """ database dependencies to support sqliteDB examples """
# from random import randrange
# from datetime import date
# import os, base64
# import json
# from sqlalchemy.exc import IntegrityError


# from flask import Flask
# from flask_sqlalchemy import SQLAlchemy


# # Setup of key Flask object (app)
# app = Flask(__name__)
# # Setup SQLAlchemy object and properties for the database (db)
# database = 'sqlite:///QA.db'  # path and filename of database
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
# app.config['SQLALCHEMY_DATABASE_URI'] = database
# app.config['SECRET_KEY'] = 'SECRET_KEY'
# db = SQLAlchemy()


# # This belongs in place where it runs once per project
# db.init_app(app)




# ''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into Python shell and follow along '''

# # Define the Post class to manage actions in 'posts' table,  with a relationship to 'users' table
# class Post(db.Model):
#     __tablename__ = 'posts'
#     __table_args__ = {'extend_existing': True}
#     # Define the Notes schema
#     id = db.Column(db.Integer, primary_key=True)
#     note = db.Column(db.Text, unique=False, nullable=False)
#     image = db.Column(db.String, unique=False)
#     # Define a relationship in Notes Schema to userID who originates the note, many-to-one (many notes to one user)
#     userID = db.Column(db.Integer, db.ForeignKey('qAs.id'))

#     # Constructor of a Notes object, initializes of instance variables within object
#     def __init__(self, id, note, image):
#         self.userID = id
#         self.note = note
#         self.image = image

#     # Returns a string representation of the Notes object, similar to java toString()
#     # returns string
#     def __repr__(self):
#         return "Notes(" + str(self.id) + "," + self.note + "," + str(self.userID) + ")"

#     # CRUD create, adds a new record to the Notes table
#     # returns the object added or None in case of an error
#     def create(self):
#         try:
#             # creates a Notes object from Notes(db.Model) class, passes initializers
#             db.session.add(self)  # add prepares to persist person object to Notes table
#             db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
#             return self
#         except IntegrityError:
#             db.session.remove()
#             return None

#     # CRUD read, returns dictionary representation of Notes object
#     # returns dictionary
#     def read(self):
#         # encode image
#         path = app.config['UPLOAD_FOLDER']
#         file = os.path.join(path, self.image)
#         file_text = open(file, 'rb')
#         file_read = file_text.read()
#         file_encode = base64.encodebytes(file_read)
        
#         return {
#             "id": self.id,
#             "userID": self.userID,
#             "note": self.note,
#             "image": self.image,
#         }


# # Define the User class to manage actions in the 'users' table
# # -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# # -- a.) db.Model is like an inner layer of the onion in ORM
# # -- b.) User represents data we want to store, something that is built on db.Model
# # -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL







# class QA(db.Model):
#     __tablename__ = 'qAs'  # table name is plural, class name is singular
#     __table_args__ = {'extend_existing': True}
#     # Define the User schema with "vars" from object
#     id = db.Column(db.Integer, primary_key=True)
#     _question = db.Column(db.String(255), unique=False, nullable=False)
#     _correctAnswer = db.Column(db.String(255), unique=False, nullable=False)
#     _incorrectAnswer1 = db.Column(db.String(255), unique=False, nullable = False)
#     _incorrectAnswer2 = db.Column(db.String(255), unique=False, nullable = False)
#     _incorrectAnswer3 = db.Column(db.String(255), unique=False, nullable = False)

#     # Defines a relationship between User record and Notes table, one-to-many (one user to many notes)
#     posts = db.relationship("Post", cascade='all, delete', backref='users', lazy=True)

#     # constructor of a User object, initializes the instance variables within object (self)
#     def __init__(self, question, correctAnswer, incorrectAnswer1, incorrectAnswer2, incorrectAnswer3):
#         self._question = question    # variables with self prefix become part of the object, 
#         self._correctAnswer = correctAnswer
#         self._incorrectAnswer1 = incorrectAnswer1
#         self._incorrectAnswer2 = incorrectAnswer2
#         self._incorrectAnswer3 = incorrectAnswer3
        


#     # a name getter method, extracts name from object
#     @property
#     def question(self):
#         return self._question
    
#     # a setter function, allows name to be updated after initial object creation
#     @question.setter
#     def question(self, name):
#         self._question = name
    
#     # a getter method, extracts email from object
#     @property
#     def correctAnswer(self):
#         return self._correctAnswer
    
#     # a setter function, allows name to be updated after initial object creation
#     @correctAnswer.setter
#     def correctAnswer(self, correctAnswer):
#         self._correctAnswer = correctAnswer
        
#     # check if uid parameter matches user id in object, return boolean
#     def is_uid(self, correctAnswer):
#         return self._correctAnswer == correctAnswer
    
#         # a getter method, extracts email from object
#     @property
#     def incorrectAnswer1(self):
#         return self._incorrectAnswer1
    
#     # a setter function, allows name to be updated after initial object creation
#     @incorrectAnswer1.setter
#     def incorrectAnswer1(self, incorrectAnswer1):
#         self._incorrectAnswer1 = incorrectAnswer1
        
#     # check if uid parameter matches user id in object, return boolean
#     def is_uid(self, incorrectAnswer1):
#         return self._incorrectAnswer1 == incorrectAnswer1
    
#     @property
#     def incorrectAnswer2(self):
#         return self._incorrectAnswer2
    
#     # a setter function, allows name to be updated after initial object creation
#     @incorrectAnswer2.setter
#     def incorrectAnswer2(self, incorrectAnswer2):
#         self._incorrectAnswer2 = incorrectAnswer2
        
#     # check if uid parameter matches user id in object, return boolean
#     def is_uid(self, incorrectAnswer2):
#         return self._incorrectAnswer2 == incorrectAnswer2
    
    
#     @property
#     def incorrectAnswer3(self):
#         return self._incorrectAnswer3
    
#     # a setter function, allows name to be updated after initial object creation
#     @incorrectAnswer3.setter
#     def incorrectAnswer3(self, incorrectAnswer3):
#         self._incorrectAnswer3 = incorrectAnswer3
        
#     # check if uid parameter matches user id in object, return boolean
#     def is_uid(self, incorrectAnswer3):
#         return self._incorrectAnswer3 == incorrectAnswer3
    

#     # output content using str(object) in human readable form, uses getter
#     # output content using json dumps, this is ready for API response
#     def __str__(self):
#         return json.dumps(self.read())

#     # CRUD create/add a new record to the table
#     # returns self or None on error
#     def create(self):
#         try:
#             # creates a person object from User(db.Model) class, passes initializers
#             db.session.add(self)  # add prepares to persist person object to Users table
#             db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
#             return self
#         except IntegrityError:
#             db.session.remove()
#             return None

#     # CRUD read converts self to dictionary
#     # returns dictionary
#     def read(self):
#         return {
#             #"id": self.id,
#             "question": self._question,
#             "correctAnswer": self._correctAnswer,
#             "incorrectAnswer1": self._incorrectAnswer1,
#             "incorrectAnswer2": self._incorrectAnswer2,
#             "incorrectAnswer3": self._incorrectAnswer3
#         }

#     # CRUD update: updates user name, password, phone
#     # returns self
#     def update(self, question="", correctAnswer="", incorrectAnswer1 = "",  incorrectAnswer2 = "",  incorrectAnswer3 = ""):
#         """only updates values with length"""
#         if len(question) > 0:
#             self.question = question
#         if len(correctAnswer) > 0:
#             self.correctAnswer = correctAnswer
#         if len(incorrectAnswer1) > 0:
#             self.incorrectAnswer1 = incorrectAnswer1
#         if len(incorrectAnswer2) > 0:
#             self.incorrectAnswer2 = incorrectAnswer2
#         if len(incorrectAnswer3) > 0:
#             self.incorrectAnswer3 = incorrectAnswer3
#         db.session.commit()
#         return self

#     # CRUD delete: remove self
#     # None
#     def delete(self):
#         db.session.delete(self)
#         db.session.commit()
#         return None


# """Database Creation and Testing """


# # Builds working data for testing
# def initQAs():
#     with app.app_context():
#         """Create database and tables"""
#         db.create_all()
#         """Tester data for table"""
#         q1 = QA(question='What is a CPU?', correctAnswer='central processing unit', incorrectAnswer1 = 'coolant protection unit', incorrectAnswer2 = 'unit processor', incorrectAnswer3 = 'man idk')
#         q2 = QA(question='What does HTML stand for?', correctAnswer='Hypertext Markup Language', incorrectAnswer1 = 'Hyperlink Maker Language', incorrectAnswer2 = 'Hyper Mail Markup', incorrectAnswer3 = 'Humphrey Man Table Lawnmower')
#         q3 = QA(question='What do we use AWS for?', correctAnswer='Deployment', incorrectAnswer1 = 'Fortnite', incorrectAnswer2 = 'Minecraft', incorrectAnswer3 = 'Java')
#         q4 = QA(question='How do we access Linux on our machines?', correctAnswer='wsl', incorrectAnswer1 = 'pylance', incorrectAnswer2 = 'Mongodb', incorrectAnswer3 = 'ReactJS')
#         q5 = QA(question='Who is our teacher?', correctAnswer='mr. yeung', incorrectAnswer1 = 'jmort223', incorrectAnswer2 = 'nathanial kim', incorrectAnswer3 = 'sabine')
#         q6 = QA(question='What does FUBAR stand for?', correctAnswer='Funked up beyond repair', incorrectAnswer1 = 'Flipped u before reprimanding', incorrectAnswer2 = 'Family Under Bad Analogies Reviewed', incorrectAnswer3 = 'Fun Ultra Boat Also Rollerblades')
#         q7 = QA(question='How many sprints have we gone through this year?', correctAnswer='23', incorrectAnswer1 = '1', incorrectAnswer2 = '18', incorrectAnswer3 = '52')
#         q8 = QA(question='Which is NOT an example of proper PII?', correctAnswer='Your car', incorrectAnswer1 = 'House address', incorrectAnswer2 = 'Bank codes', incorrectAnswer3 = 'Passwords')
#         q9 = QA(question='What do we use for our databases?', correctAnswer='SQLAlchemy', incorrectAnswer1 = 'XML', incorrectAnswer2 = 'MongoDB', incorrectAnswer3 = 'NodeJS')
#         q10 = QA(question='What would constitute as backend development?', correctAnswer='working with python to create things like a database.', incorrectAnswer1 = 'using HTML to create a webpage', incorrectAnswer2 = 'using CSS to style', incorrectAnswer3 = 'Deployment using a platform like AWS')
#         q11 = QA(question='What language is typically used in frontend development?', correctAnswer='HTML', incorrectAnswer1 = 'Python', incorrectAnswer2 = 'Assembly', incorrectAnswer3 = 'C-')
#         q12 = QA(question='What does AWS stand for?', correctAnswer='Amazon Web Services', incorrectAnswer1 = 'AWOOOOOOOGA', incorrectAnswer2 = 'Awsome Wall Seperator', incorrectAnswer3 = 'Amazonian World Services')
#         q13 = QA(question='Which of these are NOT one of the 4 roles assigned to us in CSP?', correctAnswer='Deployment Manager', incorrectAnswer1 = 'Frontend Developer', incorrectAnswer2 = 'Backend developer', incorrectAnswer3 = 'Scrum Master')
#         qAs = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12, q13]

#         """Builds sample user/note(s) data"""
#         for qA in qAs:
#             try:
#                 '''add a few 1 to 4 notes per user'''
#                 for num in range(randrange(1, 4)):
#                     note = "#### " + qA.question + " note " + str(num) + ". \n Generated by test data."
#                     qA.posts.append(Post(id=qA.id, note=note, image='ncs_logo.png'))
#                 '''add user/post data to table'''
#                 qA.create()
#             except IntegrityError:
#                 '''fails with bad or duplicate data'''
#                 db.session.remove()
#                 print(f"Records exist, duplicate email, or error: {qA.correctAnswer}")
            
# initQAs()

# class _Create(Resource):
#         def post(self):
#             ''' Read data for json body '''
#             body = request.get_json()
            
#             # ''' Avoid garbage in, error checking '''
#             # # validate name
#             question = body.get('question')
#             # if question is None or len(question) < 2:
#             #     return {'message': f'Question is missing, or is less than 2 characters'}, 400
#             # # validate correct answer
#             correctAnswer = body.get('correctAnswer')
#             # if correctAnswer is None or len(correctAnswer) == 0:
#             #     return {'message': f'answer is missing'}, 400
#             # ''' #1: Key code block, setup USER OBJECT '''
#             # uo = User(question=question, 
#             #           correctAnswer=correctAnswer)
#             # # validate incorrect answers
#             incorrectAnswer1 = body.get('incorrectAnswer1')
#             # if incorrectAnswer1 is None or len(incorrectAnswer1) == 0:
#             #     return {'message': f'answer is missing'}, 400
#             # ''' #1: Key code block, setup USER OBJECT '''
#             # uo = User(question=question, 
#             #           incorrectAnswer1=incorrectAnswer1)     
            
#             incorrectAnswer2 = body.get('incorrectAnswer2')
#             # if incorrectAnswer2 is None or len(incorrectAnswer2) == 0:
#             #     return {'message': f'answer is missing'}, 400
#             # ''' #1: Key code block, setup USER OBJECT '''
#             # uo = User(question=question, 
#             #           incorrectAnswer1=incorrectAnswer1,
#             #           incorrectAnswer2=incorrectAnswer2)
                      
#             incorrectAnswer3 = body.get('incorrectAnswer3')
#             # if incorrectAnswer3 is None or len(incorrectAnswer3) == 0:
#             #     return {'message': f'answer is missing'}, 400
#             # ''' #1: Key code block, setup USER OBJECT '''
#             uo = QA(question=question, 
#                       correctAnswer=correctAnswer,
#                       incorrectAnswer1=incorrectAnswer1,
#                       incorrectAnswer2=incorrectAnswer2,
#                       incorrectAnswer3=incorrectAnswer3)       
            
#             ''' #2: Key Code block to add user to database '''
#             # create user in database
#             qA = uo.create()
#             # success returns json of user
#             if qA:
#                 return jsonify(qA.read())
#             # failure returns error
#             return {'message': f'Processed {question}, format error'}, 400        
# class _Read(Resource):
#         def get(self):
#             qAs = QA.query.all()    # read/extract all users from database
#             random.shuffle(qAs)
#             qA = qAs[0]
#             # json_ready = [user.read() for user in users]  # prepare output in json
#             return jsonify(qA.read())  # jsonify creates Flask response object, more specific to APIs than json.dumps



